In [4]:
import numpy as np
import pandas as pd
import panel as pn
import joblib
import keras
import hvplot.pandas
import holoviews as hv

In [5]:
def Ion_Simulator(EC, Sacramento_X2,Ion, WYT, Location, month):
    X= pd.DataFrame(columns=['EC', 'Sacramento X2', 'AN', 'BN', 'C', 'D', 'W', 'OMR', 'SJRcorridor', 'SouthDelta', 'April', 'August', 'December', 'February', 'January', 'July', 'June', 'March', 'May', 'November', 'October', 'September'])

    X.loc[0,"EC"]=(EC -50)/(3500-50)
    X.loc[0,'Sacramento X2']=Sacramento_X2/100
    X.loc[0,'AN']=1 if WYT=="AN" else 0
    X.loc[0,'BN']=1 if WYT=="BN" else 0
    X.loc[0,'C']=1 if WYT=="C" else 0
    X.loc[0,'D']=1 if WYT=="D" else 0
    X.loc[0,'W']=1 if WYT=="W" else 0

    X.loc[0,'OMR']=1 if Location=="OMR" else 0
    X.loc[0,'SJRcorridor']=1 if Location=="SJRcorridor" else 0
    X.loc[0,'SouthDelta']=1 if Location=="SouthDelta" else 0

    X.loc[0,'April']=1 if month=="April" else 0
    X.loc[0,'August']=1 if month=="August" else 0
    X.loc[0,'December']=1 if month=="December" else 0
    X.loc[0,'February']=1 if month=="February" else 0
    X.loc[0,'January']=1 if month=="January" else 0
    X.loc[0,'July']=1 if month=="July" else 0
    X.loc[0,'June']=1 if month=="June" else 0
    X.loc[0,'March']=1 if month=="March" else 0
    X.loc[0,'May']=1 if month=="May" else 0
    X.loc[0,'November']=1 if month=="November" else 0
    X.loc[0,'October']=1 if month=="October" else 0
    X.loc[0,'September']=1 if month=="September" else 0

    ANN_final= keras.models.load_model(f'Models/ANN_{Ion}.h5')
    import joblib
    MLR_final=joblib.load(filename=f'Models/MLR_{Ion}.pkl')
    RT_final =joblib.load(filename=f'Models/RT_{Ion}.pkl')
    RF_final =joblib.load(filename=f'Models/RF_{Ion}.pkl')
    GB_final =joblib.load(filename=f'Models/GB_{Ion}.pkl')

    X1=X.astype(float)

    df= pd.DataFrame(columns=['MLR', 'RT','GB', 'RF', 'ANN'])
    df.loc[0,'MLR']=round(MLR_final.predict(X1)[0],2)
    df.loc[0,'RT']=round(RT_final.predict(X1)[0],2)
    df.loc[0,'GB']=round(GB_final.predict(X1)[0],2)
    df.loc[0,'RF']=round(RF_final.predict(X1)[0],2)
    df.loc[0,'ANN']=round(ANN_final.predict(X1)[0][0].astype(float),2)
    result = df


    return result




In [ ]:
# Define the input widgets for the dashboard
EC = pn.widgets.FloatSlider(name='EC', start=100, end=3500, value=200)
Sacramento_X2 = pn.widgets.FloatSlider(name='Sacramento_X2', start=0, end=100, value=90)
Ion = pn.widgets.Select(name='Ion', options=['Alkalinity','Br','Ca', 'Cl', 'K', 'Mg','Na', 'SO4','TDS'], value='Ca')
WYT = pn.widgets.Select(name='WYT', options=['AN', 'BN', 'C', 'D', 'W'], value='AN')
Location = pn.widgets.Select(name='Location', options=['OMR', 'SJRcorridor', 'SouthDelta'], value='OMR')
month = pn.widgets.Select(name='Month', options=['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'], value='September')

# Define the function to compute the ion simulator and generate the output plot
def compute_ion_simulator(event):
    # Call the Ion_Simulator function with the current widget values
    result = Ion_Simulator(EC.value, Sacramento_X2.value,Ion.value, WYT.value, Location.value, month.value)
    
    # Create a bar chart of the resulting predictions
    data={
        "Models": list(result.columns),
        'Simulation':list(result.iloc[0])
    }
    result2=pd.DataFrame(data)
    ion_name = Ion.value
    bar_chart = result2.hvplot.bar(x='Models', y='Simulation', xlabel='Machine Learning models', ylabel=f'{ion_name} [mg/L]')
    output_plot.object = bar_chart

# Create the output plot object
output_plot = pn.pane.HoloViews()

# Markdown pane to display the dashboard explanation
instructions = pn.pane.Markdown("""
## Ion Simulator Dashboard

This dashboard allows you to simulate ion concentrations based on various input parameters. Use the sliders and dropdown menus to select the desired values for EC, Sacramento_X2, Ion, WYT, Location, and Month. Then click the 'Compute' button to generate a bar chart of the predicted ion concentrations.

### Instructions:

1. Adjust the sliders and drop-down menus to select the desired input values.
2. Click the **Compute** button to run the simulation.
3. The bar chart will display the predicted ion concentrations for different machine learning models.

### Notes:

- Electrical conductivity (EC) is measured in microsiemens per centimeter (µS/cm).
- Sacramento_X2 is the percentage of Sacramento River flow that is estimated to reach the Delta.
The exact location of the Sacramento X2 point is determined by the California Department of Water Resources (DWR) based on the specific hydraulic conditions and water flows in the Sacramento River. 
The DWR uses a combination of hydrological models, flow measurements, and other data to determine the location of the Sacramento X2 point.
- The Water Year Type (WYT) is a classification of the water year based on its hydrological characteristics.
Water Year Type that includes the following categories: 1- Wet (W), 2- Critical (C), 3- Dry (D), 4- Above-Normal (AN), 5- Below-Normal (BN)
- Location refers to monitoring regions that includes: 1- Old-Middle River (OMR), 2- San Joaquin River Corridor (SJRcorridor), and 3- South Delta (SouthDelta).
- Month refers to the month of the year.
""")
    
# Define the layout of the dashboard
layout = pn.Column(
    instructions,
    pn.Row(EC, Sacramento_X2, Ion),
    pn.Row(WYT, Location, month),
    output_plot
)

# Define the event to trigger the computation of the ion simulator
compute_button = pn.widgets.Button(name='Compute')
compute_button.on_click(compute_ion_simulator)

# Create the dashboard by combining the layout
dashboard = pn.Column(layout, compute_button)

# Display the dashboard
dashboard.show()
#dashboard.servable()


Launching server at http://localhost:49992


1/1 [==============================] - 0s 55ms/step
